In [361]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [362]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [363]:
'''
Neural network for predicting live/die per passenger
'''
import torch
import torch.nn.functional as F 

class Predictor(torch.nn.Module):
    def __init__(self, num_inputs, hidden=5):
        super().__init__()
        self.hid1 = torch.nn.Linear(num_inputs, hidden)
        self.hid2 = torch.nn.Linear(hidden, hidden)
        self.output = torch.nn.Linear(hidden, 2) #binary classification
    
    def forward(self, x):
        x = F.relu(self.hid1(x))
        x = F.relu(self.hid2(x))
        return F.softmax(self.output(x))


In [364]:
'''
Data processing

what do i need to do
trim irrelevant labels
convert data to tensors
'''

def csv_to_tensors(csv_path, relevant_labels, test=False):
    train = pd.read_csv(csv_path)
    inputs_array = [] # input data, array of tensors
    labels = [] # labels to check predictions against
    passenger_ids = []
    for idx, row in train.iterrows():
        input_data = []
        for label in relevant_labels:
            if label == 'Sex':
                if row[label] == 'male':
                    input_data.append(1.0)
                else:
                    input_data.append(0.0)
            else:
                if np.isnan(float(row[label])):
                    input_data.append(0.0)
                else:
                    input_data.append(float(row[label]))
        passenger_ids.append(row['PassengerId'])
        inputs_array.append(torch.Tensor(input_data))
        if not test:
            labels.append(float(row['Survived']))
    return inputs_array, labels, passenger_ids


In [365]:
'''
training
'''
# relevant_labels = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']
relevant_labels = ['Fare', 'Pclass', 'Sex']


inputs_array, labels, _ = csv_to_tensors('./data/train.csv', relevant_labels)

model = Predictor(len(relevant_labels))
model.train()
loss_func = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for idx, inputs in enumerate(inputs_array):
    prediction = model(inputs)
    target = None
    if labels[idx] == 0.0:
        target = torch.Tensor([1.0, 0.0])
    else:
        target = torch.Tensor([0.0, 1.0])
    
    # print(inputs)
    print(prediction)
    loss = loss_func(prediction, target)
    print(target)
    print(loss)
    loss.backward()
    optimizer.step()



/var/folders/74/3w92brss03v2m756k4vq_zc00000gn/T/ipykernel_11148/150891609.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.output(x))


tensor([0.2495, 0.7505], grad_fn=<SoftmaxBackward0>)
tensor([1., 0.])
tensor(1.3883, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.0013, 0.9987], grad_fn=<SoftmaxBackward0>)
tensor([0., 1.])
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.2005, 0.7995], grad_fn=<SoftmaxBackward0>)
tensor([0., 1.])
tensor(0.2238, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.0063, 0.9937], grad_fn=<SoftmaxBackward0>)
tensor([0., 1.])
tensor(0.0064, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.2390, 0.7610], grad_fn=<SoftmaxBackward0>)
tensor([1., 0.])
tensor(1.4312, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.2340, 0.7660], grad_fn=<SoftmaxBackward0>)
tensor([1., 0.])
tensor(1.4525, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.0099, 0.9901], grad_fn=<SoftmaxBackward0>)
tensor([1., 0.])
tensor(4.6116, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([0.1039, 0.8961], grad_fn=<SoftmaxBackward0>)
tensor([1., 0.])
tensor(2.2645, grad_fn=<BinaryCrossEntropyBackward0>)
tensor([

In [366]:
# relevant_labels = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']

inputs_array, labels, passenger_ids = csv_to_tensors('./data/test.csv', relevant_labels, test=True)

model.eval()
# print(passenger_ids)
predictions = []
# print(labels)
for idx, inputs in enumerate(inputs_array):
    prediction = torch.argmax(model(inputs))
    predictions.append(prediction.item())
    print(prediction)
    # if(prediction == 1):
        # print("out : " + str(passenger_ids[idx]) + " " + str(prediction))

output = pd.DataFrame()
output['PassengerId'] = passenger_ids
output['Survived'] = predictions
output.to_csv('./data/my_predictions.csv', index=None)

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


/var/folders/74/3w92brss03v2m756k4vq_zc00000gn/T/ipykernel_11148/150891609.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.output(x))
